ссылка на гитхаб репозиторий

https://github.com/theorcos239/async_ppo.git



# Коротко о структуре

model.py - файл, в котором хранится класс основной действующей модели: свёрточная нейронная сеть 
с двумя головами для actor и critic

learner.py - файл, содержащий класс, реализующий PPO, с соответствующими методами по сбору роллаутов и обучению

rollout_buffer.py - реализация класса роллаута

worker_process.py - функция процесса асинхронного воркера для сбора роллаутов

learner_process.py - процесс обучения центральной модели 






# Параметры эксперимента

workers_count = 4

rollout_size = 2048

lr = 3e-4

batch_size = 256

n_epochs = 4
                        
gamma = 0.99 

delta = 0.95

clip_eps = 0.2
                              
entropy_coef = 0.01 

value_coef = 0.5

Обучение проходило на GPU A100 в течение 1 часа на эксперимент.


# Результаты

<img src="./rl_ppo_results/time_reward.png">

Из графика выше видно, что системы с низкими frequency(частота обновления) обучаются быстрее, но при этом менее стабильно. В некотором смысле можно объянить тем, что они чаще обучаются на "свежих" данных, в отличие отб например, систем с frequency = 10, которые сдерживаются старыми данными при обучении. Отсюда и относительная стабильность последних (дольше смотрят на старые данные).

<img src="./rl_ppo_results/timesteps_reward.png">

<img src="./rl_ppo_results/updates_reward.png">

Аналогичная история для графиков timesteps/reward и updates/reward

<img src="./rl_ppo_results/time_updates.png">

Выше тоже закономерный результат. Из определения процесса понятно.

<img src="./rl_ppo_results/time_timesteps.png">

А вот это уже интересный результат. Оказалось, что при max_queue_size = 10 происходит больше взаимодействий со средой, чем обычно. 

<img src="./rl_ppo_results/time_queue_size.png">

Из графика зависимости размера очереди от времени видно, что очередь во всех случаях обычно полна и редко изменяется на 1. Наверное, это связано с тем, что нейронная сеть всё таки не успевает обрабатывать всё, что приходит от воркеров. 

<img src="./rl_ppo_results/time_difference.png">

Видно, что постепенно версия роллаутов догоняет версию центральной нейросети, а затем держится примерно на одном уровне. Также довольно логично, что чем выше размер очереди, тем больше отставание, ведь старые данные в более длинной очереди задерживаются. 